In [1]:
import sklearn as skl
import numpy as np
import pandas as pd
from typing import Tuple, List, TypeVar, Dict, Any, Union

### Part A

In [2]:
# HERE YOU WILL WRITE CODE TO TEST A NUMBER OF PREDICTORS
# AND FINALLY CHOOSE AND TRAIN THE PREDICTOR THAT YOU WILL BE USING FOR PART B

## Initially prodding the data to find out about it

Once again, the lack of any feature names is particularly unhelpful.

In [3]:
_all_data: pd.DataFrame = pd.read_csv("CE802_P3_Data.csv")
_all_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F28,F29,F30,F31,F32,F33,F34,F35,F36,Target
0,271.78,-21.92,UK,-10.80,1899.57,17.98,-21749.82,91.94,855.61,10.01,...,2098.80,500.16,-6325.53,1.36,-33.14,177.34,-141.97,101.52,5,189.03
1,202.54,43.82,USA,-16.94,1941.57,-9.16,-27668.04,93.50,975.44,7.29,...,1668.70,434.97,-6172.05,2.59,-58.87,87.48,-154.11,623.22,6,187.17
2,220.26,88.90,Europe,-18.76,2298.12,-18.38,-11548.56,65.16,1114.28,12.05,...,2604.56,252.93,-10132.68,2.94,-40.89,271.00,-279.84,284.96,2,1016.24
3,141.00,140.72,Europe,-19.86,-133.32,-57.00,-16200.96,-14.00,910.12,4.54,...,2595.56,154.83,-7862.04,0.86,-117.03,201.66,-153.93,532.19,4,-141.18
4,165.04,2.74,Europe,-21.34,3077.07,-20.50,-25683.06,29.08,216.24,10.10,...,1066.80,316.68,-6093.81,3.59,-63.84,211.82,-182.34,373.14,5,33.17


In [4]:
_all_data.dtypes

F1        float64
F2        float64
F3         object
F4        float64
F5        float64
F6        float64
F7        float64
F8        float64
F9        float64
F10       float64
F11        object
F12       float64
F13       float64
F14       float64
F15       float64
F16       float64
F17       float64
F18       float64
F19       float64
F20       float64
F21       float64
F22       float64
F23         int64
F24       float64
F25       float64
F26       float64
F27       float64
F28       float64
F29       float64
F30       float64
F31       float64
F32       float64
F33       float64
F34       float64
F35       float64
F36         int64
Target    float64
dtype: object

In [5]:
_all_data_inputs: pd.DataFrame = _all_data.loc[:,_all_data.columns!="Target"]
_all_data_targets:pd.DataFrame = _all_data.loc[:,"Target"]


It looks like I'll need to convert F3 and F11 into a less awkward format.
Of course, there's the simple approach of doing one-hot encoding, but...

In [6]:
print(_all_data_inputs["F3"].unique())

print(_all_data_inputs["F11"].unique())

['UK' 'USA' 'Europe' 'Rest']
['Very low' 'Medium' 'Low' 'Very high' 'High']


As F11 indicates values with some sequential nature to them
(very low->low->medium->high->very high), I could get away with converting them
to ints. Which is what I am going to do.

However, for F3, I should probably just replace them with some one-hot encoding instead.


In [7]:

def df_f11_and_f3_to_sensible(df: pd.DataFrame) -> pd.DataFrame:

    f11dict = { "F11": {
        "Very low": 0,
        "Low": 1,
        "Medium": 2,
        "High": 3,
        "Very high": 4
    }}

    df.replace(f11dict, inplace=True)

    df = pd.get_dummies(df, columns=["F3"])

    return df

all_data = df_f11_and_f3_to_sensible(pd.read_csv("CE802_P3_Data.csv"))

_all_data_inputs = df_f11_and_f3_to_sensible(_all_data_inputs)

c:\python3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
print(_all_data_inputs.dtypes)
print("")
_all_data_inputs.head()

F1           float64
F2           float64
F4           float64
F5           float64
F6           float64
F7           float64
F8           float64
F9           float64
F10          float64
F11            int64
F12          float64
F13          float64
F14          float64
F15          float64
F16          float64
F17          float64
F18          float64
F19          float64
F20          float64
F21          float64
F22          float64
F23            int64
F24          float64
F25          float64
F26          float64
F27          float64
F28          float64
F29          float64
F30          float64
F31          float64
F32          float64
F33          float64
F34          float64
F35          float64
F36            int64
F3_Europe      uint8
F3_Rest        uint8
F3_UK          uint8
F3_USA         uint8
dtype: object



,F1,F2,F4,F5,F6,F7,F8,F9,F10,F11,...,F31,F32,F33,F34,F35,F36,F3_Europe,F3_Rest,F3_UK,F3_USA
0,271.78,-21.92,-10.80,1899.57,17.98,-21749.82,91.94,855.61,10.01,0,...,1.36,-33.14,177.34,-141.97,101.52,5,0,0,1,0
1,202.54,43.82,-16.94,1941.57,-9.16,-27668.04,93.50,975.44,7.29,2,...,2.59,-58.87,87.48,-154.11,623.22,6,0,0,0,1
2,220.26,88.90,-18.76,2298.12,-18.38,-11548.56,65.16,1114.28,12.05,2,...,2.94,-40.89,271.00,-279.84,284.96,2,1,0,0,0
3,141.00,140.72,-19.86,-133.32,-57.00,-16200.96,-14.00,910.12,4.54,1,...,0.86,-117.03,201.66,-153.93,532.19,4,1,0,0,0
4,165.04,2.74,-21.34,3077.07,-20.50,-25683.06,29.08,216.24,10.10,0,...,3.59,-63.84,211.82,-182.34,373.14,5,1,0,0,0


Much better.

Anyway, here's a helper function that'll be useful later on,
pretty much wrapping that dataframe reading/splitting/reformatting into a nice
and simple function for ease of use later on.

In [9]:
def np_data_and_targets(df: pd.DataFrame = pd.read_csv("CE802_P3_Data.csv"), already_formatted_f11_f3: bool = False, targetname: str = "Target") -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts dataframe into a couple of numpy ndarrays for the data without the labels,
    and the labels by themselves.
    Also handles the f11 and f3 formatting.
    :param df: the Dataframe
    :param already_formatted_f11_f3: set this to true if we've already replaced f11 and f3 with sensible formatting.
    :param targetname: The name of the column holding the targets
    :return: tuple of [ndarray of the values without the targets, just the class labels]
    """

    inputs:  np.ndarray = df.loc[:,df.columns != targetname].to_numpy() if already_formatted_f11_f3 else df_f11_and_f3_to_sensible(df.loc[:,df.columns != targetname]).to_numpy()
    outputs: np.ndarray =  df.loc[:,targetname].to_numpy()

    return inputs, outputs

# Setting up a hold-out validation set for later

Nested cross-validation moment.

This validation set consists of 300 random items from the dataset.

In [10]:
from sklearn.model_selection import KFold

validation_cv: KFold = KFold(n_splits=5, shuffle=True)
all_train_indices, validation_indices = [i for i in validation_cv.split(_all_data_inputs, _all_data_targets)][0]

validation_df: pd.DataFrame = all_data.iloc[validation_indices]
tests: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(validation_df, True)
test_data: np.ndarray = tests[0]
test_labels: np.ndarray = tests[1]

train_df: pd.DataFrame = all_data.iloc[all_train_indices]
trains: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(train_df, True)
train_data: np.ndarray = trains[0]
train_labels: np.ndarray = trains[1]

print("validation")
print(validation_df.head())
print(validation_df.shape)
print("train")
print(train_df.head())
print(train_df.shape)


validation
        F1      F2     F4       F5      F6        F7     F8       F9    F10  \
2   220.26   88.90 -18.76  2298.12  -18.38 -11548.56  65.16  1114.28  12.05   
3   141.00  140.72 -19.86  -133.32  -57.00 -16200.96 -14.00   910.12   4.54   
4   165.04    2.74 -21.34  3077.07  -20.50 -25683.06  29.08   216.24  10.10   
5   247.64   26.68 -30.24  1868.22 -121.16 -10192.29  -9.68   428.42   9.07   
16  145.36   85.52 -15.06  3658.14  -29.56 -24281.61  95.70   470.17   9.28   

    F11  ...     F32     F33     F34     F35  F36   Target  F3_Europe  \
2     2  ...  -40.89  271.00 -279.84  284.96    2  1016.24          1   
3     1  ... -117.03  201.66 -153.93  532.19    4  -141.18          1   
4     0  ...  -63.84  211.82 -182.34  373.14    5    33.17          1   
5     2  ... -132.50  223.08 -244.28  459.16    5   277.33          0   
16    3  ...   -5.95  159.40 -145.85  349.06    6   387.40          0   

    F3_Rest  F3_UK  F3_USA  
2         0      0       0  
3         0      

# Setting up some grid-search nested cross validation pipelines

Once again, we're using HalvingGridSearchCV for the param_grid cross-validating search,
because it's supposed to be faster than GridSearchCV, and speed like that is beneficial
for my fragile ~~lack of~~ mental health.

Unlike the classification tasks, there won't be any scaling happening here. This is because
this particular task expects a real value to be output, instead of an abstract
label or probability of a label being given. For classification tasks, scaling is
generally a lot more appropriate, as abstract inputs can safely lead to abstract outputs.
But, again, no abstract outputs means that abstract inputs could get problematic.
And I don't want to have to deal with that headache.

We're still using KNNImputer to impute missing features based on the features held by 'nearby' individuals,
as that still seems like the most rational imputation method available given the circumstances,
and is still a lot more elegant than simply discarding everything with a null value.

Due to this task involving linear regression and the occasional problems of data leakage involved with
linearly regressing on a high-dimensional dataset, I have attempted to compensate for this via the
usage of a
PolynomialFeatures preprocessor. Of course, I have no idea which parameter configuration
for it is optimal, so I am trying to work this out via the grid search cross validation.
Yes, the PolynomialFeatures does have the end result of 'addressing'
the dimensionality issue by, uhh, adding more dimensonality.
But this extra dimensionality mostly consists of multiplying the values
in two of the 30-something dimensions together, in turn adding more dimensions in which
each value has an impact, and, in theory, reducing the risk of certain values getting overlooked when
they shouldn't.
And if a value is still utterly unimportant, it can still be overlooked as usual!

One other thing I should also mention is that I'm using the R^2 score
to evaluate the performance of the parameter configurations for the grid search cv stuff.
R^2 basically aggregates the differences between each prediction and the average expected value/the
actual value, and represents them as a value that's <=1. If it's below 0, that means that the
regressor isn't even getting the expected values correct, which is a major red flag. But anything
above 0 is basically an improvement over random guessing.

Finally, you might be wondering why the error score I'm using (for when there's an oopsie
and the estimator completely fails to work) is a finite negative value. In short, I wanted to use
negative infinity, but that caused warning messages to get plastered over the outputs in the notebook,
complaining about the negative infinity value, so I had to compromise and put a stupidly big
(in the negative sense) finite negative value in there for these situations instead. I'm working on
the theory that if there is a situation that causes such a stupid value to be returned legitimately for
the score, chances are that the problem of 'non-erroneous but utterly terrible result discarded in
favour of an erroneous result' will be the least of everyone's concerns.


In [11]:
from sklearn.base import RegressorMixin
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.metrics import r2_score, make_scorer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from math import inf

set_config(display="diagram")

R = TypeVar('R', bound=RegressorMixin)


In [32]:
def h_grid_searcher(
        regressor: R,
        param_grid: Dict[str, List[Any]],
        train_data: np.ndarray,
        train_targets: np.ndarray,
        k_fold: KFold = KFold(n_splits=5, shuffle=True, random_state=42)
) -> HalvingGridSearchCV:


    # last minute refactor to ensure that the Automatic Relevance Detection
    # works in a reasonable legnth of time.

    #param_grid["imputer__weights"] = ["distance","uniform"]
    #param_grid["poly__degree"] = [2,3]
    #param_grid["poly__interaction_only"] = [True, False]
    #param_grid["poly__include_bias"] = [True, False]

    pl: Pipeline = Pipeline([
        ("imputer",KNNImputer(add_indicator=True, weights="distance")),
        ("poly", PolynomialFeatures()),
        ("regressor",regressor)
    ])

    h_grid_search: HalvingGridSearchCV = HalvingGridSearchCV(
        estimator=pl,
        param_grid=param_grid,
        factor=3,
        cv=k_fold,
        scoring=make_scorer(r2_score),
        refit=True,
        verbose=1,
        n_jobs=-1,
        error_score=-1000000000000
        # I wanted to make this error score negative infinity, however, doing so caused a lot of
        # particularly unsightly warning messages to appear.

        # So, to save everyone involved from having to look at at a buttload of them with a buttload of numbers in them,
        # I'm just setting this to an incredibly low finite number which should be rather hard to reach.
        # And if this score (or an even lower score) somehow is reached legitimately, chances are that
        # the legitimate score being lower than the error score will be the least of one's concerns.
    )

    h_grid_search.fit(train_data, train_targets)

    return h_grid_search

In [33]:
def nested_h_grid_searcher(
        regressor: R,
        param_grid: Dict[str, List[Any]],
        train_data: np.ndarray,
        train_targets: np.ndarray,
        t_df: pd.DataFrame,
        kfold_splits: int = 6
) -> Dict[HalvingGridSearchCV, float]:

    h_grid_search_dicts: Dict[HalvingGridSearchCV, float] = {}

    kf: KFold = KFold(n_splits=kfold_splits, shuffle=True)

    for train_indices, test_indices in kf.split(train_data, train_targets):
        tr: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(t_df.iloc[train_indices],True)
        te: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(t_df.iloc[test_indices],True)

        current_search: HalvingGridSearchCV = h_grid_searcher(
            regressor,
            param_grid,
            tr[0],
            tr[1],
            KFold(n_splits=kfold_splits-1, shuffle=False)
        )

        current_score: float = current_search.score(te[0],te[1])

        h_grid_search_dicts[current_search] = current_score

    return h_grid_search_dicts

# Linear Regression

There isn't much that I can do in terms of hyper-parameter search for simple linear regression,
but, here goes.

In [14]:

from sklearn.linear_model import LinearRegression

linear_regression_searched_dict: Dict[
    HalvingGridSearchCV, float
] = nested_h_grid_searcher(
    LinearRegression(),
    {
        "poly__degree": [2,3],
        "poly__interaction_only": [True, False],
        "poly__include_bias": [True, False],
        "imputer__weights": ["distance","uniform"]
    },
    train_data,
    train_labels,
    train_df
)

linreg_searched: HalvingGridSearchCV = max(
    linear_regression_searched_dict.keys(), key=lambda k: linear_regression_searched_dict[k]
)

best_linreg: Tuple[HalvingGridSearchCV, float] = (
    linreg_searched, linear_regression_searched_dict[linreg_searched]
)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 111
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 16
n_resources: 111
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 1
n_candidates: 6
n_resources: 333
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 2
n_candidates: 2
n_resources: 999
Fitting 5 folds for each of 2 candidates, totalling 10 fits
n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 111
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 16
n_resources: 111
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 1
n_candidates: 6
n_resources: 333
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 2
n_candidates: 2
n_resources: 999
Fitting 5 folds for each of 2 candidates, totalling 10 fits
n_iterations: 3
n_required_ite

In [15]:
print(best_linreg)

print(best_linreg[1])

linreg_searched.best_estimator_


(HalvingGridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                    error_score=-1000000000000,
                    estimator=Pipeline(steps=[('imputer',
                                               KNNImputer(add_indicator=True)),
                                              ('poly', PolynomialFeatures()),
                                              ('regressor',
                                               LinearRegression())]),
                    n_jobs=-1,
                    param_grid={'imputer__weights': ['distance', 'uniform'],
                                'poly__degree': [2, 3],
                                'poly__include_bias': [True, False],
                                'poly__interaction_only': [True, False]},
                    refit=<function _refit_callable at 0x00000292BE6995E0>,
                    scoring=make_scorer(r2_score), verbose=1), -7.0379606171048525)
-7.0379606171048525


Pipeline(steps=[('imputer', KNNImputer(add_indicator=True, weights='distance')),
                ('poly',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('regressor', LinearRegression())])

# RANdom SAmple Consensus (RANSAC) regression

This is an adaption of the linear regression method, of the 'robust regressor' variety.
This means it basically copes with outliers a lot better than a simple linear regressor would,
although, in spaces of higher dimensionality (such as this problem), it apparently can struggle.
It deals with outliers by, in short, completely ignoring them.

It's an iterative approach, randomly sampling several items from the population and
seeing if they're 'valid' data or not, trying to fit a model to that random subset,
classifiying them as inliers/outliers appropriately, saving the current model as the 'best' model
if it has more inliers than the previous model (if same inliers, saving if it has a better score),
and repeating for a certain number of trials, or until reaching a certain threshold of inliers
or score threshold.

For this situation, I have defined a score threshold of 1 (being the largest possible R2 score)
for a premature stop. Of course, I could have defined a score threshold of 0.95 or 0.99
etc as being good enough, but I figured that as there was very little chance of reaching those
R2 values anyway, I may as well go all-in and aim for an R2 value of 1.


In [17]:
from sklearn.linear_model import RANSACRegressor

rs_samples: List[Union[int, None]] = [None, 2, 4, 8]
rs_max_trials: List[int] = [80,100,120]
rs_residual: List[Union[float, None]] = [None,100, 1000, 10000]

rs_param_dict: Dict[str, List[Any]] = {
    "regressor__min_samples": rs_samples,
    "regressor__max_trials": rs_max_trials,
    "regressor__residual_threshold": rs_residual,
    "poly__degree": [2,3],
    "poly__interaction_only": [True, False],
    "poly__include_bias": [True, False],
    "imputer__weights": ["distance","uniform"]
}

ransac_regression_searched_dict: Dict[
    HalvingGridSearchCV, float
] = nested_h_grid_searcher(
    RANSACRegressor(base_estimator=LinearRegression(), stop_score=1),
    rs_param_dict,
    train_data,
    train_labels,
    train_df
)

ransac_searched: HalvingGridSearchCV = max(
    ransac_regression_searched_dict.keys(), key=lambda k: ransac_regression_searched_dict[k]
)

best_ransac: Tuple[HalvingGridSearchCV, float] = (
    ransac_searched, ransac_regression_searched_dict[ransac_searched]
)

print(best_ransac)


n_iterations: 5
n_required_iterations: 7
n_possible_iterations: 5
min_resources_: 10
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 768
n_resources: 10
Fitting 5 folds for each of 768 candidates, totalling 3840 fits
----------
iter: 1
n_candidates: 256
n_resources: 30
Fitting 5 folds for each of 256 candidates, totalling 1280 fits
----------
iter: 2
n_candidates: 86
n_resources: 90
Fitting 5 folds for each of 86 candidates, totalling 430 fits
----------
iter: 3
n_candidates: 29
n_resources: 270
Fitting 5 folds for each of 29 candidates, totalling 145 fits
----------
iter: 4
n_candidates: 10
n_resources: 810
Fitting 5 folds for each of 10 candidates, totalling 50 fits
n_iterations: 5
n_required_iterations: 7
n_possible_iterations: 5
min_resources_: 10
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 768
n_resources: 10
Fitting 5 folds for each of 768 candidates, totalling 3840 fits
----------
ite

In [18]:
print(best_ransac[1])

ransac_searched.best_estimator_




-0.08791271925419597


Pipeline(steps=[('imputer', KNNImputer(add_indicator=True)),
                ('poly', PolynomialFeatures(interaction_only=True)),
                ('regressor',
                 RANSACRegressor(base_estimator=LinearRegression(),
                                 max_trials=80, min_samples=4,
                                 residual_threshold=100, stop_score=1))])

And now, time for something completely different.

# Relevance Vector Machine - Bayesian Ridge Regression with Automatic Relevance Detection (ARD)

This attempts to use Bayesian logic to produce a probabilistic model of the regression problem,
inherently also regularizing the data being fed into it. The regularization is based
on three 'alpha' and 'lambda' parameters, with one alpha being responsible
for declaring the precision of the noise, one lambda setting the precision of the weights,
and the other two of each being responsible for declaring how the initial alpha/lambda get
adjusted over time. In the case of the ARD regressor, only the supplementary alpha and lambda
need to be declared, as the ARD regressor will attempt to estimate the main alpha/lambda itself.

It attempts to perform this est

One thing that must be taken into account with this variety of regression is that it takes a
rather long time to train. Therefore, in the interests of completing this investigation within a
reasonable length of time, I had to perform some last-minute refactors of the param search code
within the cross-validator methods to remove the automatic investigation of different
KNNImputer and PolynomialFeatures parameters, moving the code putting that stuff into the
grid_param_dicts into the definition of their grid_param_dicts themselves (the end result
being that those code cells still do the same overall thing, and this investigation doesn't
have that bit of investigation tacked onto it).

Furthermore, I initially planned to look into the effects of different values for
`tol` (used to determine when the algorithm considers convergence to have happened)
and `threshold_lambda` (used for pruning), but, due to time constraints (both in terms of
deadline time constraints, and time I can physically spend waiting for this to run on a
computer, with decent hardware, which is acting particularly sluggish (to the point of being unable
to do any other work in the meantime on it) due to having to process this), those investigations
had to be unceremoniously scrapped, instead focusing the investigation on changing the powers
of the alpha and lambda (1 and 2) hyper-parameters.


In [34]:
from sklearn.linear_model import ARDRegression, BayesianRidge

ard_tol: List[float] = [1e-2, 1e-3, 1e-4]
ard_alpha_lambda: List[float] = [1e-5, 1e-6, 1e-7]
ard_thresh_lambda: List[float] = [7500, 10000, 12500]

ard_param_dict: Dict[str, List[Any]] = {
    #"regressor__tol": ard_tol,
    "regressor__alpha_1" : ard_alpha_lambda,
    "regressor__alpha_2" : ard_alpha_lambda,
    "regressor__lambda_1" : ard_alpha_lambda,
    "regressor__lambda_2" : ard_alpha_lambda,
    #"regressor__threshold_lambda": ard_thresh_lambda
}

# had to comment out the investigations into the tolerance values and pruning threshold lambdas
# in the interests of being able to get this grid_param_search done within a half-reasonable amount of time.


ard_grid_search_dict: Dict[
    HalvingGridSearchCV, float
] = nested_h_grid_searcher(
    ARDRegression(n_iter=300, verbose=True),
    ard_param_dict,
    train_data,
    train_labels,
    train_df
)

ard_searched: HalvingGridSearchCV = max(
    ard_grid_search_dict.keys(), key=lambda k: ard_grid_search_dict[k]
)

best_ard: Tuple[HalvingGridSearchCV, float] = (
    ard_searched, ard_grid_search_dict[ard_searched]
)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 12
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 81
n_resources: 12
Fitting 5 folds for each of 81 candidates, totalling 405 fits
----------
iter: 1
n_candidates: 27
n_resources: 36
Fitting 5 folds for each of 27 candidates, totalling 135 fits
----------
iter: 2
n_candidates: 9
n_resources: 108
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 3
n_candidates: 3
n_resources: 324
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 4
n_candidates: 1
n_resources: 972
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Converged after 17 iterations
n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 12
max_resources_: 1000
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 81
n_resources: 12
Fitting 5 folds for each of 81 candidates, totalling 405 fits
-

In [35]:
print(best_ard)

print(best_ard[1])

ard_searched.best_estimator_


(HalvingGridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                    error_score=-1000000000000,
                    estimator=Pipeline(steps=[('imputer',
                                               KNNImputer(add_indicator=True,
                                                          weights='distance')),
                                              ('poly', PolynomialFeatures()),
                                              ('regressor',
                                               ARDRegression(verbose=True))]),
                    n_jobs=-1,
                    param_grid={'regressor__alpha_1': [1e-05, 1e-06, 1e-07],
                                'regressor__alpha_2': [1e-05, 1e-06, 1e-07],
                                'regressor__lambda_1': [1e-05, 1e-06, 1e-07],
                                'regressor__lambda_2': [1e-05, 1e-06, 1e-07]},
                    refit=<function _refit_callable at 0x00000292BE6995E0>,
                    scor

Pipeline(steps=[('imputer', KNNImputer(add_indicator=True, weights='distance')),
                ('poly', PolynomialFeatures()),
                ('regressor',
                 ARDRegression(alpha_2=1e-07, lambda_2=1e-07, verbose=True))])

# Attempting to run each estimator on my held-out test set

In [36]:
tests: Tuple[np.ndarray, np.ndarray] = np_data_and_targets(validation_df, True)
test_data: np.ndarray = tests[0]
test_labels: np.ndarray = tests[1]

print(len(test_data))
print(len(test_labels))

300
300


In [37]:
linreg_score: float = linreg_searched.score(test_data, test_labels)
print(linreg_score)

-2.497730104411637


In [38]:
ransac_score: float = ransac_searched.score(test_data, test_labels)
print(ransac_score)


-0.10247452886073871


In [39]:
ard_score: float = ard_searched.score(test_data, test_labels)
print(ard_score)

0.6946717708240002


In [40]:
all_regressors: List[Tuple[HalvingGridSearchCV, float, str]] = [
    (linreg_searched, linreg_score, "Linear regression"),
    (ransac_searched, ransac_score, "RANdom SAmple Consensus regression"),
    (ard_searched, ard_score, "Relevance Vector Machine - Bayesian Ridge Regression with Automatic Relevance Detection")
]

best_regressor: Tuple[HalvingGridSearchCV, float, str] = max(
    all_regressors, key=lambda psn: psn[1]
)

print("The best regressor is...")
print(best_regressor[2])

print("")
print(f"Overall R score of {best_regressor[1]}")

The best regressor is...
Relevance Vector Machine - Bayesian Ridge Regression with Automatic Relevance Detection

Overall R score of 0.6946717708240002


### Part B

In [41]:
# HERE YOU WILL USE THIS TEMPLATE TO SAVE THE PREDICTIONS ON THE TEST SET

# Load the test data
test_df = pd.read_csv('CE802_P3_Test.csv')

final_data, final_targets = np_data_and_targets(pd.read_csv("CE802_P3_Test.csv"))



predicted = best_regressor[0].predict(final_data)
# CHANGE HERE -- use your previously trained predictor and apply it to test_data
# (test_data can be modified if needed but make sure you don't change the order of the rows)...

# Replace the last (empty) column with your prediction
test_df.iloc[:,-1] = predicted



# Save to the destination file
test_df.to_csv('CE802_P3_Test_Predictions.csv', index=False, float_format='%.8g')

# IMPORTANT!! Make sure only the last column has changed
assert pd.read_csv('CE802_P3_Test.csv').iloc[:,:-1].equals(pd.read_csv('CE802_P3_Test_Predictions.csv').iloc[:,:-1])

c:\python3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
